In [1]:
import gensim, os, re
from elasticsearch import Elasticsearch

ImportError: No module named 'gensim'

In [2]:
txt_dir="./txt/"

# create stopword lists - *customize/manually create*
stop_words = set{[
        ...
    ]}
# create the list by doing the following and just add.

def clean_text(txt):
    # get a string of raw text and tokenize them
    # make sure we dont have stop words
    # make sure length > 1
    
    clean_words = []
    tokens = txt.split()
    for w in tokens:
        if w not in stop_words and re.search( r"^[A-Za-z].*]$", w): # make sure starts and ends with letter and is not stop word, order matters (regexp are expensive)
            w = re.sub(r"[^A-Za-z-]", "", w) # make sure the word contains letters or hyphen
            if len(w) > 1:
                clean_words.append(w.lower())
    return clean_words

# elastic has momentum right now
    # schema on read
es = Elasticsearch()
index = "arxiv"

if es.indices.exists(index):
    es.indices.delete(index)

es.indices.create(index)

for file_name in os.listdir(txts_dir):
    with open(txtss + file_name, "r") as f:
        txt = f.read()
        clean_text = " ".join(clean_text(text))
        name = file_name.replace(".txt", "")
        body = {"name" : name, "text"}
        es.index(index = index, doc_type = "file", body = body)

SyntaxError: invalid syntax (<ipython-input-2-c4f149431865>, line 4)

In [3]:
# find the number of topics/clusters for you
# nonparametric, number of clusters grow as you give it more data

In [ ]:
docs = []

for f in os.listdir(txts_dir):
    with open(txts_dir + f, "r") as f:
        docs.append(f.read())
        
texts = [clean_text(d) for d in docs]

# create dictionary
id2word = gensim.corpora.Dictionary(texts)
id2word.filter_extremes(no_below=10, keep_n = 100000) # if word appears fewer than 10 times, disregard; keep 100k words in our vocab
id2word.compactify()

# we want to make sure we can save, so we can get our trained model and apply it
id2word.save("hdpDictionary")

# corpus in gensim
id2freq=[id2word.doc2bow(t) for t in texts] # turn to bag of words, does not take context into account

# now we can train our model
# faster than LDA, hdp infers for you k; "no advantage"
# implementation from one papers
hdp = gensim.model.hdpmodel(corpus=id2freq, id2word=id2word)
hdp.save("hdpModel")

# passed corpus array of bag of words for each text
# passed dictionary id to word so ethat gensim can represent word as integer instead of string

topics_file = "hdp_topics.txt"
topics = hdp.show_topics(topics = -1, topn = 15) # grab all topics (-1) and top 15 most relevant words from each topic

with open(topics_file, "w") as f:
    for topic in topics:
        f.write("{}\n".format(topic))


print "done processing"

# more or less topics, tweak alpha